# Memory

In this notebook we'll walk through how [LangChain](https://python.langchain.com/en/latest/index.html#) thinks about memory. Memory involves keeping a concept of state around throughout a user’s interactions with an language model. A user’s interactions with a language model are captured in the concept of ChatMessages, so this boils down to ingesting, capturing, transforming and extracting knowledge from a sequence of chat messages. There are many different ways to do this, each of which exists as its own memory type.

List of main relevant tutorials/blog posts

* [Getting started with chains (Langchain)](https://python.langchain.com/en/latest/modules/chains/getting_started.html)
* [LangChain for LLM Application Development (DeepLearning.AI)](https://learn.deeplearning.ai/langchain/)
* [Briggs and Ingham (2023)](https://www.pinecone.io/learn/langchain/). LangChain AI Handbook.

# 1 - Introduction

By default, Chains and Agents are stateless, meaning that they treat each incoming query independently (as are the underlying LLMs and chat models). However, in some applications (chatbots being a GREAT example) it is highly important to remember previous interactions, both at a short term but also at a long term level. The **Memory** does exactly that.

LangChain provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular and useful regardless of how they are used. Secondly, LangChain provides easy ways to incorporate these utilities into chains.

Four different kinds of memory:

* Conversation buffer memory
* Conversation entity memory
* Conversation knowledge graph memory 
* VectorStore-backed memory

We can use multiple memories at one time. For example, we can use conversation memory + entity memory to recall individuals.

We can also use other types of memory no directly implemented in langchain. For instance, we can store the conversation in a conventional database (such as a key-value store or SQL).

# 2 - Conversation buffer memory

Conversation buffer memory is the simplest form of memory. It just involves keeping a buffer of all prior messages. There are different types:

* `ConversationBufferMemory`. Allows for storing of messages and then extracts the messages in a variable.
* `ConversationBufferWindowMemory`. Keeps a list of the interactions of the conversation over time. It only uses the last $K$ interactions.
* `ConversationTokenBufferMemory`. Keeps a buffer of recent interactions in memory, and uses token lenght rather than the number of interactions to determine when to flush iteractions.
* `ConversationSummaryMemory`. Creates a summary of the conversation over time and appends it to the prompts with the `System` tag. This approach keeps the last interaction in memory an summarize the previous ones. So, when a new interaction comes in, the previous one is combined with the summary.

In [1]:
import openai
import os
from langchain.llms import AzureOpenAI

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.environ["OPENAI_API_KEY"]

engine = "Davinci003"
max_tokens = 1000

llm = AzureOpenAI(deployment_name=engine)
llm.openai_api_key = openai.api_key
llm.openai_api_base = openai.api_base 
llm.max_tokens = max_tokens

## 2.1 - `ConversationBufferMemory`

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [3]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


" Hi Andrew! It's nice to meet you. My name is AI. What brings you here today?"

In [4]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hi Andrew! It's nice to meet you. My name is AI. What brings you here today?
Human: What is 1+1?
AI:

> Finished chain.


' 1+1 is equal to 2.'

In [5]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hi Andrew! It's nice to meet you. My name is AI. What brings you here today?
Human: What is 1+1?
AI:  1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


" Your name is Andrew. It's nice to meet you."

#### Access memory buffer

In [6]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI:  Hi Andrew! It's nice to meet you. My name is AI. What brings you here today?
Human: What is 1+1?
AI:  1+1 is equal to 2.
Human: What is my name?
AI:  Your name is Andrew. It's nice to meet you.


In [7]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI:  Hi Andrew! It's nice to meet you. My name is AI. What brings you here today?\nHuman: What is 1+1?\nAI:  1+1 is equal to 2.\nHuman: What is my name?\nAI:  Your name is Andrew. It's nice to meet you."}

#### Manually append to memory

In [8]:
memory.save_context({"input": "What's up?"}, 
                    {"output": "Not much, just hanging"})

In [9]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI:  Hi Andrew! It's nice to meet you. My name is AI. What brings you here today?
Human: What is 1+1?
AI:  1+1 is equal to 2.
Human: What is my name?
AI:  Your name is Andrew. It's nice to meet you.
Human: What's up?
AI: Not much, just hanging


## 2.2 - `ConversationBufferWindowMemory`

`ConversationBufferWindowMemory` keeps a list of the interactions of the conversation over time. It only uses the last $K$ interactions. Ths can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [10]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [11]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


" Hi Andrew, my name is AI. It's nice to meet you. What can I do for you today?"

In [12]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hi Andrew, my name is AI. It's nice to meet you. What can I do for you today?
Human: What is 1+1?
AI:

> Finished chain.


' 1+1 is equal to 2. Is there anything else I can help you with?'

In [13]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI:  Hi Andrew, my name is AI. It's nice to meet you. What can I do for you today?
Human: What is 1+1?
AI:  1+1 is equal to 2. Is there anything else I can help you with?
Human: What is my name?
AI:

> Finished chain.


" I'm sorry, I don't know your name."

In [15]:
print(memory.buffer)

[HumanMessage(content='Hi, my name is Andrew', additional_kwargs={}, example=False), AIMessage(content=" Hi Andrew, my name is AI. It's nice to meet you. What can I do for you today?", additional_kwargs={}, example=False), HumanMessage(content='What is 1+1?', additional_kwargs={}, example=False), AIMessage(content=' 1+1 is equal to 2. Is there anything else I can help you with?', additional_kwargs={}, example=False), HumanMessage(content='What is my name?', additional_kwargs={}, example=False), AIMessage(content=" I'm sorry, I don't know your name.", additional_kwargs={}, example=False)]


As you can see the buffer is the same, but the prompt is artificially trimmed when sent to the model to emulate the window approach.

## 2.3 - `ConversationTokenBufferMemory`

`ConversationTokenBufferMemory` mantiene un búfer de interacciones recientes en la memoria y utiliza la longitud de tokens en lugar del número de interacciones para determinar cuándo descartar interacciones.

Al mirar el código, puede parecer contraintuitivo que necesitemos pasar el objeto LLM como argumento. La razón detrás de esto es que diferentes LLMs cuentan los tokens de manera diferente.

**Nota:** Debes instalar el paquete `tiktoken` para contar los tokens de GPT

In [19]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

We can see how the "memory window" changes when we change the `max_token_limit`

In [20]:
memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## 2.4 - `ConversationSummaryBufferMemory`

Instead of limiting the number of exchanges or tokens, let's write a summary of the conversation so far and let that be the memory. This approach keeps the last interaction in memory an summarize the previous ones. So, when a new interaction comes in, the previous one is combined with the summary.

In [1]:
from langchain.memory import ConversationSummaryBufferMemory

# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

NameError: name 'llm' is not defined

In [22]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and asks what is on the schedule today. The AI responds casually with "Cool".\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.'}

In [23]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)
conversation.predict(input="What would be a good demo to show?")

' A good demo to show would be one that highlights the latest features of the LLM and how they can be used in a real-world setting. You could also demonstrate how the AI capabilities can be integrated with other systems.'

In [24]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and asks what is on the schedule today. The AI responds casually with "Cool" and goes on to explain that there is a meeting at 8am with the product team to prepare a powerpoint presentation, then from 9am-12pm there is time to work on the LangChain project, and then at noon lunch with a customer who is driving from over an hour away to understand the latest in AI, and to bring the laptop to show the latest LLM demo.\nHuman: What would be a good demo to show?\nAI:  A good demo to show would be one that highlights the latest features of the LLM and how they can be used in a real-world setting. You could also demonstrate how the AI capabilities can be integrated with other systems.'}

In [28]:
conversation.predict(input="Do I have time to meet Harry at 10am? (according to my schedule)")

' I recommend that you create between 8 and 10 slides for your demo. It is important to keep your presentation concise, but still have enough details to make the audience understand the features of the LLM and how it can be used in a real-world setting. Depending on the length of the demo, you may need to adjust the number of slides accordingly.'

In [29]:
memory.load_memory_variables({})

{'history': 'System: \nThe human greets the AI and asks what is on the schedule today. The AI responds casually with "Cool" and goes on to explain that there is a meeting at 8am with the product team to prepare a powerpoint presentation, then from 9am-12pm there is time to work on the LangChain project, and then at noon lunch with a customer who is driving from over an hour away to understand the latest in AI. The AI suggests that a good demo to show would be one that highlights the latest features of the LLM and how they can be used in a real-world setting, and possibly demonstrate how the AI capabilities can be integrated with other systems. When asked if there was time to meet Harry at 10am, the AI confirmed that there was one hour from 9am to 10am to work on the LangChain project, so the human would be able to meet Harry at 10am.\nHuman: How many slides do you recommend for my demo?\nAI:  I recommend that you create between 8 and 10 slides for your demo. It is important to keep you

In [ ]:
conversation.predict(input="How many slides do you recommend for my demo?")

In [ ]:
memory.load_memory_variables({})

# 3 - Entity memory

This memory module extract information on entities (using LLMs) and builds up its knowledge about that entity over time (also using LLMs).

**Note:** It would be interesting to understand how this works with well-known entities like "Mount Everest"

In [34]:
from langchain.memory import ConversationEntityMemory

memory = ConversationEntityMemory(llm=llm)
_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)
memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

We can ask the model to search for information about entities:

In [31]:
memory.load_memory_variables({"input": 'who is Sam'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [32]:
memory.load_memory_variables({"input": 'who is Deven'})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Deven': 'Deven is working on a hackathon project with Sam.',
  'Sam': 'Sam is working on a hackathon project with Deven.'}}

In [33]:
memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

## 3.1 - Automatic usage in a Conversation

In the following example, we have 4 entities:
* Sam (unknown)
* Deven (unknown)
* Langchain (unknown in 2021, i.e., last model's data)
* Everest (known)

When generating the summary of the entities we can see how the model incorporates extra info to "known" entities given the given context, we can also see how it modifies all info about entities accordingly.

In [57]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

conversation.memory.clear()

In [58]:
conversation.predict(input="Deven & Sam are working on a hackathon project")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' What kind of project are they working on?'

In [59]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam.',
 'Sam': 'Sam is working on a hackathon project with Deven.'}

In [60]:
conversation.predict(input="They are trying to add more complex memory structures to Langchain")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' Interesting! What kind of memory structures are they trying to add?'

In [61]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam to add more complex memory structures to Langchain.',
 'Sam': 'Sam is working on a hackathon project with Deven, attempting to add more complex memory structures to Langchain.',
 'Langchain': 'Langchain is a project that is trying to add more complex memory structures.'}

In [62]:
conversation.predict(input="Deven & Sam are going to climb the Everest")



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

' That sounds like an ambitious goal! What challenges do you think they might face while attempting to climb the Everest?'

In [63]:
conversation.memory.entity_store.store

{'Deven': 'Deven is working on a hackathon project with Sam to add more complex memory structures to Langchain, and is attempting to climb the Everest.',
 'Sam': 'Sam is working on a hackathon project with Deven, attempting to add more complex memory structures to Langchain, and is also planning to climb the Everest.',
 'Langchain': 'Langchain is a project that is trying to add more complex memory structures.',
 'Everest': 'Everest is a mountain that Deven and Sam are attempting to climb.'}

# 4 - Knowledge graph memory

Use a knowledge graph to recreate memory

In [87]:
from langchain.memory import ConversationKGMemory

# Recreate a fake conversation
memory = ConversationKGMemory(llm=llm)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is one of my high-school friends"}, {"output": "hi Sam!"})
memory.save_context({"input": "Sam's favourite color is blue"}, {"output": "it is a nice color"})
memory.save_context({"input": "thanks :)"}, {"output": "you are welcome"})

memory.load_memory_variables({"input": 'who is sam'})

{'history': "On Sam: Sam is one of Human's high-school friends. Sam favourite color blue."}

**Note:** I have noticed that in order to properly show the system message on the following example, you have to execute it twice (don't know why)

In [89]:
# Recreate a fake conversation and return messages instead of model "thoughts"
memory = ConversationKGMemory(llm=llm, return_messages=True)
memory.save_context({"input": "say hi to sam"}, {"output": "who is sam"})
memory.save_context({"input": "sam is one of my high-school friends"}, {"output": "hi Sam!"})
memory.save_context({"input": "Sam's favourite color is blue"}, {"output": "it is a nice color"})
memory.save_context({"input": "thanks :)"}, {"output": "you are welcome"})

memory.load_memory_variables({"input": 'who is sam'})

{'history': [SystemMessage(content='On Sam: Sam is one of my high-school friends. Sam has favorite color. Sam favorite color is blue.', additional_kwargs={})]}

## 4.1 - Automatic usage in a conversation

**Note:** Interestingly, I tested this same example with the default prompt and the model hallucinated when asked about Will (it said that Will studied in Standford for 5 years and so on).

In [94]:
from langchain.prompts.prompt import PromptTemplate

template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)
conversation_with_kg = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [95]:
conversation_with_kg.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing well, just learning more about the world and the different data I can access. How about you?"

In [96]:
conversation_with_kg.predict(input="My name is James and I'm helping Will. He's an engineer.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: My name is James and I'm helping Will. He's an engineer.
AI:

> Finished chain.


" Hi James. It's nice to meet you. I'm an AI. It sounds like Will is a very interesting person. What type of engineering does he specialize in?"

In [97]:
conversation_with_kg.predict(input="What do you know about Will?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

On Will: Will is engineer.

Conversation:
Human: What do you know about Will?
AI:

> Finished chain.


" Will is an engineer. I don't know any more information about him."

# 5 - VectorStore-backed memory

`VectorStoreRetrieverMemory` stores memories in a VectorDB and queries the top-K most "salient" docs every time it is called. This differs from most of the other Memory classes in that it doesn’t explicitly track the order of interactions.

In this case, the “docs” are previous conversation snippets. This can be useful to refer to relevant pieces of information that the AI was told earlier in the conversation.

In [ ]:
from langchain.embeddings import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="TextEmbeddingAda002",
    openai_api_version="2023-05-15",
    openai_api_key=config.get("OPENAI_API_KEY", "")
)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

loader = TextLoader("./state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
batch_size = 16

# Split the documents into batches
document_batches = [docs[i:i + batch_size] for i in range(0, len(docs), batch_size)]

# Initialize the FAISS db with the first batch of documents
initial_batch = document_batches.pop(0)
db = FAISS.from_documents(initial_batch, embeddings)

# Add the remaining batches
for batch in document_batches:
    db.add_documents(batch)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs_back = db.similarity_search(query)
docs_back